<a href="https://colab.research.google.com/github/znafi/Cmput-267/blob/main/ass5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Enable widgets
from google.colab import output
output.enable_custom_widget_manager()
import ipywidgets as widgets

# Install otter-grader
%pip install -q otter-grader==6.1.6

# Download the tests directory from the course website (this will be used by otter-grader)
!wget -q https://vladtkachuk4.github.io/machinelearning1/assignments/Fall2025/ass5/tests.zip -O tests.zip

# Unzip the tests directory, forcing overwriting of existing files
!unzip -qo tests.zip -d .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.8/118.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 8.8 MB/s eta 0:00:00


In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

# CMPUT 267 - Machine Learning I
# Assignment 5 - Evaluating Predictors

## Assignment Instructions and Information
For this assignment, we will be using Google Colab. If you are new to Google Colab, you can find a brief introduction at the following link: [Google Colab Introduction](https://colab.research.google.com/notebooks/intro.ipynb).

**Important:** Before you start working on this notebook, make sure to save a copy to your own Google Drive. To do this, go to `File` -> `Save a copy in Drive`.

If you do not save a copy, you will not be able to save any changes you make.


### Submitting your assignment
Once you have completed the assignment, please submit your work as a `.ipynb` file on eClass in the "Assignment 5" section. To download your Colab notebook in the required format, follow these steps:

1. Save your notebook to ensure all changes are preserved.
2. Navigate to `File` -> `Download` -> `Download .ipynb`.

Make sure to save your notebook before downloading it!

### Questions and Autograding

Each section contains questions for you to solve, marked with the subsection heading "Question X.Y" (ex: the first problem is "Question 1.1").

For each question your solution must go within the following designated code block:

```python
### YOUR CODE HERE ###

######################
```

All questions will be autograded using [Otter-Grader](https://otter-grader.readthedocs.io/en/latest/).
The first two code cells in this notebook install the Otter-Grader package and download the test cases.
You should run these cells, otherwise the autograder will not work.

At the end of each question there is code that runs the autograder. For example, in Question 1.1 the code `grader.check("q1_1")` runs the autograder.
If you pass all the test cases for a question (ex: Question 1.1), you will see the following output:

**q1_1** passed!

If you do not pass all the test cases for a question, you will see which test cases you did not pass along with their corresponding error messages.

There are both public and private test cases. You only have access to the public test cases. This means that if you pass all the test cases in this notebook, you have passed all the public test cases.

After you submit the assignment, we will also run the private test cases. The public test cases account for 50% of the total mark, while the private test cases make up the remaining 50%. Therefore, if you pass all the test cases in the notebook, you are guaranteed a mark of at least 50%.

After each question description, the number of points (marks) the question is worth is indicated.
This is the total number of points for both the public and private test cases.
For each question, the public test cases are worth 50% of the points.


### Math Rendering Issues
If you're encountering issues with math rendering in your notebook, particularly small squares replacing math symbols, this is a common problem, especially for users of Google Chrome. Fortunately, there's an easy fix.

Hold down `Ctrl` on your keyboard and click on the small square. Then select: Math Settings -> Math Renderer -> Common HTML.

## Introduction

Welcome to Assignment 5! In this assignment, you will explore evaluating predictors.
We will be focusing on the methods discussed in Chapter 7 of the [course notes](https://vladtkachuk4.github.io/machinelearning1/notes.pdf).

This assignment will guide you through the following sections:

1. [Visualizing Different Errors](#part-1-visualizing-different-errors)
1. [Picking the Best Polynomial Predictor](#part-2-picking-the-best-polynomial-predictor)
1. [Regularization](#part-3-regularization)

You can quickly navigate to each section by opening the table of contents on the left side of the screen.

Let's get started!


# Part 1: Visualizing Different Errors

In this assignment we will use synthetic data that we generate ourselves.
This means that we have access to the feature-label distribution $\mathbb{P}_{\boldsymbol{X}, Y}$.
This is useful because it allows us to calculate the expected loss $L$ of our predictors.

To make visualizing things easier, we will use a 1-dimensional features (i.e. $d=1$).
Thus,
$$\mathcal{X} = \mathbb{R}^{d+1} = \mathbb{R}^2 \quad \text{and} \quad \mathcal{Y} = \mathbb{R}$$

Since we are working in the regression setting we will be using the squared loss function
$$\ell(\hat{y}, y) = (\hat{y} - y)^2.$$

In this section our goal will be to visualize and build a better understanding of the different error terms we encounter when trying to minimize $\mathbb{E}[L(\mathcal{A}(D))]$.

To get started, lets import all the necessary libraries for this assignment

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from ipywidgets import interactive_output
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

# Set a fixed random seed for reproducibility
random_seed = 42

Now, lets visualize the best possible predictor
$$f_\text{Bayes} = \arg\min_{f \in \{f | f: \mathcal{X} \to \mathcal{Y}\}} L(f).$$
and what $\mathbb{P}_{\boldsymbol{X}, Y}$ looks like by ploting 1000 samples from it.

One way to interpret $f_\text{Bayes}$ is as the predictor that best fits infinitely many samples from $\mathbb{P}_{\boldsymbol{X}, Y}$.
Since we can not plot infinitely many samples we only plotted finitely many (1000 in this case).

In [ ]:
# @title Plot

# Define the coefficients for f_Bayes
coefficients = [1, 7, -6, 1.2, 0.1, -0.02, 0.001, 0, -0.0002]

# Define a polynomial function
def polynomial(coeffs):
    def poly_func(x):
        return sum(c * x**i for i, c in enumerate(coeffs))
    return poly_func

# Define f_Bayes
f_bayes = polynomial(coefficients)

np.random.seed(random_seed)

# Number of data points
num_points_plot = 1000

# Generate X values uniformly between 0 and 4
X_plot = np.random.uniform(0, 4, num_points_plot)

# Generate noise from a Gaussian distribution with mean 0 and variance 1
noise_plot = np.random.normal(0, 1, num_points_plot)

# Calculate Y values using f_Bayes and adding noise
Y_plot = f_bayes(X_plot) + noise_plot

# Generate x values for plotting
x = np.linspace(0, 4, 400)

# Define the function to update the plot based on the checkbox value
def update_plot_f_bayes(show_plot):
    if show_plot:
        # Plot the generated data points
        plt.figure(figsize=(10, 6))
        plt.scatter(X_plot, Y_plot, alpha=0.5, label=r'Samples from $\mathbb{P}_{X, Y}$')
        plt.plot(x, f_bayes(x), color='black', label=r'$f_{Bayes}$', linewidth=3)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.ylim(0.2, 7.2)
        plt.xlim(-0.2, 4.2)
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        clear_output()
        # plt.close()

# Create a checkbox for showing/hiding the plot
show_plot_checkbox_f_bayes = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to update the plot based on the checkbox value
interactive_plot_f_bayes = widgets.interactive_output(update_plot_f_bayes, {'show_plot': show_plot_checkbox_f_bayes})

# Display the checkbox and the plot
display(show_plot_checkbox_f_bayes, interactive_plot_f_bayes)

Now we would like to see what the best predictor in a polynomial degree $p$ function class $\cal{F}_p$ looks like.
In particular, the predictor
$$f^*_p = \arg\min_{f \in \cal{F}_p} L(f)$$
where
$$\cal{F}_p = \{f | f: \mathcal{X} \to \mathcal{Y}, \text{ where } f(\boldsymbol{x}) = \phi_p(\boldsymbol{x})^\top \boldsymbol{w}, \text{ and } \boldsymbol{w} \in \mathbb{R}^{\bar p}\}$$
To do this we will need an implemention of the polynomial feature map $\phi_p$ for various values of $p$.
To test your understanding we will only ask you to implement $\phi_3$.

After your implementation, we provide you with a more general implementation of $\phi_p$ for any $p$, that will be used for the rest of the assignment.

**Do not just copy the code from `phi_p` into the cell where you are asked to implement $\phi_3$. You will get zero marks for that.**

### Question 1.1

Implement `phi_3`.

_Points:_ 8

In [ ]:
def phi_3(x):
    """
    Transforms a single feature vector into a polynomial of degree 3 feature vector.

    Parameters:
    x (numpy array): A single feature vector of size d+1.

    Returns:
    poly_features (numpy array): A transformed feature vector of size (d+1)*(d+2)*(d+3)/6.
    """
    # Calculate the size of the transformed feature vector
    d = len(x) - 1
    size = (d+1) * (d+2) * (d+3) // 6

    ### YOUR CODE HERE ###

    poly_features = np.random.rand(size) # this is a placeholder, replace it with the correct value

    ######################

    return poly_features


In [ ]:
grader.check("q1_1")

Below is the `phi_p` function that implements $\phi_p$ for any $p$, which we will use for the rest of the assignment.
We directly add normalization the the end of the `phi_p` function, so that we don't have to constantly normalize the features after applying the `phi_p` function each time.
The purpose of normalization was discussed in assignment 4.

In [ ]:
def phi_p(X, p):
    """
    Transforms the input feature matrix X into polynomial features of degree p and normalizes them.

    Parameters:
    X (numpy.ndarray): Input feature matrix where each row is a feature vector.
    p (int): Degree of the polynomial features.

    Returns:
    poly_features (numpy.ndarray): Transformed and normalized polynomial features.
    """
    # Create PolynomialFeatures instance with the desired degree
    poly = PolynomialFeatures(degree=p, include_bias=True)

    # Transform the feature matrix to polynomial features
    poly_features = poly.fit_transform(X[:, 1:])

    # Normalize the polynomial features, excluding the first column
    poly_features[:, 1:] = (poly_features[:, 1:] - np.mean(poly_features[:, 1:], axis=0)) / np.std(poly_features[:, 1:], axis=0)

    return poly_features

Lets plot $f^*_p$ for various values of $p$ and compare it to $f_\text{Bayes}$.

You can use the slider to select different values of $p$.

Notice how as $p$ increases, $f^*_p$ is able to better approximate $f_\text{Bayes}$.
This is because $\cal{F}_p$ is a richer function class as $p$ increases, and for large enough $p$, we will even have that $f_\text{Bayes} \in \cal{F}_p$.

In [ ]:
# @title Plot

np.random.seed(random_seed)
# num_points = 1000000
num_points = 10000
X_exp = np.random.uniform(0, 4, num_points)
noise_exp = np.random.normal(0, 1, num_points)
Y_exp = f_bayes(X_exp) + noise_exp

# Define the function to update the plot based on the selected degree and checkbox value
def update_plot_f_star(degree, show_plot):
    if show_plot:
        # Fit the polynomial to the data
        w_star = np.polyfit(X_exp, Y_exp, degree)

        # Generate y values for the fitted polynomial
        f_star = np.polyval(w_star, x)

        # Plot the generated data points and the polynomial fit
        plt.figure(figsize=(10, 6))
        plt.plot(x, f_bayes(x), color='black', label=r'$f_{Bayes}$', linewidth=3)
        plt.plot(x, f_star, color='green', linestyle='-', label=fr'$f^*_{degree}$', linewidth=3)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.ylim(0.2, 7.2)
        plt.xlim(-0.2, 4.2)
        plt.legend(loc='upper left')
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create sliders for the polynomial degree and a checkbox for showing/hiding the plot
degree_slider_f_star = widgets.IntSlider(value=1, min=1, max=9, step=1, description='Degree p')
show_plot_checkbox_f_star = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to update the plot based on the slider and checkbox values
interactive_plot_f_star = widgets.interactive_output(update_plot_f_star, {'degree': degree_slider_f_star, 'show_plot': show_plot_checkbox_f_star})

# Display the sliders, checkbox, and the plot
display(degree_slider_f_star, show_plot_checkbox_f_star, interactive_plot_f_star)

Now, we would like to plot the best predictor based on a dataset of size $n$.
In particular, the predictor
$$\hat f_{p} = \arg\min_{f \in \cal{F}_p} \hat L(f).$$
Recall that the closed form linear regression learner outputs exactly this predictor.
Since you have already implemented it in the previous assignment, we will provide you with the implementation of the learner below.

In [ ]:
def closed_form_learner(X, Y):
    '''
    Solves linear regression using the closed form solution.

    Parameters:
    X (numpy array): Feature matrix of size (n, d+1), where n is the number of samples
                     and d is the number of features. The first column should be all 1s.
    Y (numpy array): Target vector of size (n, 1).

    Returns:
    predictor (function): A function that takes a feature vector and returns a predicted value.
    w_hat (numpy array): The weights calculated using the closed form solution.
    '''

    A = X.T @ X
    b = X.T @ Y
    w_hat = np.linalg.pinv(A) @ b

    def predictor(x):
        return x @ w_hat

    return predictor, w_hat

We can add $\hat f_{p}$ to our previous plot and see how it compares.

We have added 2 more sliders so that you can change the size of the dataset $n$ and the random seed used to generate the dataset.
Changing the random seed should be thought of as getting an entirely new dataset of size $n$.

The orange dots are the data points from the dataset.
You should notice that as $n$ increases, $\hat f_{p}$ is a better approximation of $f^*_p$.
You should also notice that if you change the random seed, the dataset changes and thus $\hat f_{p}$ changes.
Another thing to notice is that if you increase the degree of the polynomial $p$, more samples $n$ are needed for $\hat f_p$ to approximate $f^*_p$ well, and if you change the random seed, $\hat f_{p}$ changes more drastically.

In [ ]:
# @title Plot

# Define the function to update the plot based on the selected degree, n, and random seed
def update_plot_f_hat(degree, n, random_seed, show_plot):
    if show_plot:
        # Fit the polynomial to the data
        w_star = np.polyfit(X_exp, Y_exp, degree)

        # Generate y values for the fitted polynomial
        f_star = np.polyval(w_star, x)

        # Generate new train dataset
        np.random.seed(random_seed)
        X_train_raw = np.random.uniform(0, 4, n)
        X_train = np.c_[np.ones(n), X_train_raw]
        noise_train = np.random.normal(0, 1, n)
        Y_train = f_bayes(X_train_raw) + noise_train

        # Transform the feature matrix to polynomial features
        X_train_poly = phi_p(X_train, degree)

        # Fit the model using closed form solution
        f_hat, w_hat = closed_form_learner(X_train_poly, Y_train)

        # Generate y values for the fitted polynomial from closed_form_learner
        f_hat_values = f_hat(phi_p(np.c_[np.ones(x.shape[0]), x], degree))

        # Plot the generated data points and the polynomial fit
        plt.figure(figsize=(10, 6))
        plt.plot(x, f_bayes(x), color='black', label=r'$f_{Bayes}$', linewidth=3)
        plt.plot(x, f_star, color='green', linestyle='-', label=fr'$f^*_{degree}$', linewidth=3)
        plt.plot(x, f_hat_values, color='orange', linestyle='--', label=fr'$\hat f_{degree}$ with n={n}', linewidth=3)
        plt.scatter(X_train_raw, Y_train, color='orange', alpha=0.5, label='Dataset')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.ylim(0.2, 7.2)
        plt.xlim(-0.2, 4.2)
        plt.legend(loc='upper left')
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create sliders for the polynomial degree, n, and random seed
degree_slider_f_hat = widgets.IntSlider(value=1, min=1, max=9, step=1, description='Degree p')
n_slider_f_hat = widgets.IntSlider(value=10, min=1, max=200, step=1, description='n')
random_seed_slider_f_hat = widgets.IntSlider(value=1, min=1, max=10, step=1, description='Random Seed', style={'description_width': 'initial'})
show_plot_checkbox_f_hat = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to update the plot based on the slider values
interactive_plot_f_hat = widgets.interactive_output(update_plot_f_hat,
                                                    {'degree': degree_slider_f_hat,
                                                     'n': n_slider_f_hat,
                                                     'random_seed': random_seed_slider_f_hat,
                                                     'show_plot': show_plot_checkbox_f_hat})

# Display the sliders and the plot
display(degree_slider_f_hat, n_slider_f_hat, random_seed_slider_f_hat, show_plot_checkbox_f_hat, interactive_plot_f_hat)

In class we also discussed the expected predictor
$$\bar f_p(x) = \mathbb{E}[\hat f_{D, p}(X)|X = x],$$
which is the predictor we get after taking the expected value of $\hat f_{D, p}(x)$ for each $x \in [0,4]$ over all possible datasets.
We have added $D$ to the subscript of $\hat f_p$ here to emphasize that it is a function of the random variable $D$, thus it it also a random variable and that is what the expectation is taken over.

Since it is difficult to calculate this expected value we plot an estimate of $\bar f_p$ based on 1000 datasets.
What you should see is that $\bar f_p$ is approximately equal to $f^*_p$.
Indeed, if we could calculate $\bar f_p$ exactly then you would see that is equal to $f^*_p$.
This is a property that holds when we use the closed form linear regression learner.
We will see in [Part 3](#part-3-regularization) that this property does not hold when we use regularization.

In [ ]:
# @title Plot

# Define the function to update the plot based on the selected degree, n, and random seed
def update_plot_f_bar(degree, n, random_seed, show_plot):
    if show_plot:
        # Fit the polynomial to the data
        w_star = np.polyfit(X_exp, Y_exp, degree)

        # Generate y values for the fitted polynomial
        f_star = np.polyval(w_star, x)

        # Generate new train dataset
        np.random.seed(random_seed)
        X_train_raw = np.random.uniform(0, 4, n)
        X_train = np.c_[np.ones(n), X_train_raw]
        noise_train = np.random.normal(0, 1, n)
        Y_train = f_bayes(X_train_raw) + noise_train

        # Transform the feature matrix to polynomial features
        X_train_poly = phi_p(X_train, degree)

        # Fit the model using closed form solution
        f_hat, w_hat = closed_form_learner(X_train_poly, Y_train)

        # Generate y values for the fitted polynomial from closed_form_learner
        f_hat_values = f_hat(phi_p(np.c_[np.ones(x.shape[0]), x], degree))

        # Calculate the average predictor
        num_datasets = 1000
        f_bar_values = np.zeros_like(x)

        for _ in range(num_datasets):
            X_train_raw = np.random.uniform(0, 4, n)
            X_train = np.c_[np.ones(n), X_train_raw]
            noise_train = np.random.normal(0, 1, n)
            Y_train = f_bayes(X_train_raw) + noise_train

            X_train_poly = phi_p(X_train, degree)
            f_bar, w_bar = closed_form_learner(X_train_poly, Y_train)
            f_bar_values += f_bar(phi_p(np.c_[np.ones(x.shape[0]), x], degree))

        f_bar_values /= num_datasets

        # Plot the generated data points and the polynomial fit
        plt.figure(figsize=(10, 6))
        plt.plot(x, f_bayes(x), color='black', label=r'$f_{Bayes}$', linewidth=3)
        plt.plot(x, f_star, color='green', linestyle='-', label=fr'$f^*_{degree}$', linewidth=3)
        plt.plot(x, f_hat_values, color='orange', linestyle='--', label=fr'$\hat f_{degree}$ with n={n}', linewidth=3)
        plt.plot(x, f_bar_values, color='blue', linestyle='-.', label=fr'$\bar f_{degree}$ over {num_datasets} datasets', linewidth=3)
        plt.scatter(X_train_raw, Y_train, color='orange', alpha=0.5, label='Dataset')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.ylim(0.2, 7.2)
        plt.xlim(-0.2, 4.2)
        plt.legend(loc='upper left')
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create sliders for the polynomial degree, n, and random seed
degree_slider_f_bar = widgets.IntSlider(value=1, min=1, max=9, step=1, description='Degree p')
n_slider_f_bar = widgets.IntSlider(value=10, min=1, max=200, step=1, description='n')
random_seed_slider_f_bar = widgets.IntSlider(value=1, min=1, max=10, step=1, description='Random Seed', style={'description_width': 'initial'})
show_plot_checkbox_f_bar = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to update the plot based on the slider values
interactive_plot_f_bar = widgets.interactive_output(update_plot_f_bar,
                                                    {'degree': degree_slider_f_bar,
                                                     'n': n_slider_f_bar,
                                                     'random_seed': random_seed_slider_f_bar,
                                                     'show_plot': show_plot_checkbox_f_bar})

# Display the sliders and the plot
display(degree_slider_f_bar, n_slider_f_bar, random_seed_slider_f_bar, show_plot_checkbox_f_bar, interactive_plot_f_bar)

Now we would like to plot the expected and estimated losses of the predictors we have discussed above.
To do this you will need to first implement `estimated_loss`, which is the estimate of expected loss, defined as
$$\hat L(f) = \frac{1}{n} \sum_{i=1}^n \ell(f(x_i), y_i).$$
Remember that we are using the squared loss.
Also, `X` is a matrix, where each row is a feature vector, and `Y` is a vector of labels, similar to the previous assignment.

### Question 1.2
Implement `estimated_loss`.

_Points:_ 4

In [ ]:
def estimated_loss(f, X, Y):
    """
    Estimate the expected loss for a given predictor function.

    Parameters:
    f (function): Predictor function that takes an input feature matrix X and returns predicted values.
    X (numpy.ndarray): Input feature matrix.
    Y (numpy.ndarray): Target vector.

    Returns:
    loss (float): Estimate of the expected loss.
    """

    ### YOUR CODE HERE ###

    loss = 0 # this is a placeholder, replace it with the correct value

    ######################

    return loss

In [ ]:
grader.check("q1_2")

Now we can plot $L(f_\text{Bayes})$, $L(f^*_p)$, $\mathbb{E}[L(\hat f_{D, p})]$, and $\mathbb{E}[\hat L(\hat f_{D, p})]$ for various values of $p$.

You should see the same trends as we discussed in class.
In particular, the irreducible error $L(f_\text{Bayes})$ is the lowest; however, it is not zero because of the noise in the data.
The approximation error $L(f^*_p)- L(f_\text{Bayes})$ should decrease as $p$ increases.
The estimation error $\mathbb{E}[L(\hat f_{D, p})] - L(f^*_p)$ should increase as $p$ increases, which can explained due to $\mathbb{E}[L(\hat f_{D, p})] - \mathbb{E}[\hat L(\hat f_{D, p})]$ getting larger as $p$ increases.

You can also use the slider to change the size of the dataset $n$.
You should notice that as $n$ increases, the estimation error decreases.

In [ ]:
# @title Plot

# degrees = [1, 2, 3, 4, 5, 6, 7]
degrees = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Define the function to update the plot based on the value of n and show_plot checkbox
def update_plot_true_loss(n, show_plot):
    if show_plot:
        np.random.seed(random_seed)
        # Number of different train datasets to use
        num_datasets = 100

        # Calculate the loss for each polynomial degree using closed_form_learner
        L_f_star = []
        L_f_hat = []
        L_hat_f_hat = []

        # Calculate the loss for f_bayes
        L_f_bayes = estimated_loss(f_bayes, X_exp, Y_exp)

        for degree in degrees:
            L_hat_f_hat_temp = []
            L_f_hat_temp = []

            coeffs = np.polyfit(X_exp, Y_exp, degree)
            f_star = lambda x: np.polyval(coeffs, x)

            # Calculate the estimated loss
            L = estimated_loss(f_star, X_exp, Y_exp)
            L_f_star.append(L)

            for _ in range(num_datasets):
                # Generate new train dataset
                X_train_raw = np.random.uniform(0, 4, n)
                X_train = np.c_[np.ones(n), X_train_raw]
                noise_train = np.random.normal(0, 1, n)
                Y_train = f_bayes(X_train_raw) + noise_train

                # Transform the feature matrix to polynomial features
                X_train_poly = phi_p(X_train, degree)

                # Fit the model using closed form solution
                f_hat, w_hat = closed_form_learner(X_train_poly, Y_train)

                # Calculate the estimated loss on train data
                L_hat = estimated_loss(f_hat, X_train_poly, Y_train)
                L_hat_f_hat_temp.append(L_hat)

                # Calculate the estimated loss on the entire dataset
                X_poly = phi_p(np.c_[np.ones(X_exp.shape[0]), X_exp], degree)
                L = estimated_loss(f_hat, X_poly, Y_exp)
                L_f_hat_temp.append(L)

            # Take the average of the losses
            L_hat_f_hat.append(np.mean(L_hat_f_hat_temp))
            L_f_hat.append(np.mean(L_f_hat_temp))

        # Plot the estimated loss as a function of polynomial degrees and the closed form learner loss
        plt.figure(figsize=(10, 6))
        plt.plot(degrees, L_f_star, color='green', marker='o', label=r'$L(f^*_p)$', linewidth=3, markersize=10)
        plt.plot(degrees, L_f_hat, color='orange', marker='x', linestyle='--', label=r'$\mathbb{E}[L(\hat{f}_{D, p})]$', linewidth=3, markersize=10)
        plt.plot(degrees, L_hat_f_hat, color='purple', marker='x', linestyle='--', label=r'$\mathbb{E}[\hat{L}(\hat{f}_{D, p})]$', linewidth=3, markersize=10)
        plt.axhline(y=L_f_bayes, color='black', linestyle='-', label=r'$L(f_{Bayes})$', linewidth=3)
        plt.xlabel('Polynomial Degree p')
        plt.ylim(0.2, 3)
        plt.legend(loc='lower left')
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a slider for the variable n and a checkbox for showing/hiding the plot
n_slider_true_loss = widgets.IntSlider(value=30, min=10, max=200, step=2, description='n')
show_plot_checkbox_true_loss = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to update the plot based on the slider and checkbox values
interactive_plot_true_loss = widgets.interactive_output(update_plot_true_loss, {'n': n_slider_true_loss, 'show_plot': show_plot_checkbox_true_loss})

# Display the slider, checkbox, and the plot
display(n_slider_true_loss, show_plot_checkbox_true_loss, interactive_plot_true_loss)

Recall that we would like to select the predictor that minimizes the expected loss.
If we had access to the above plot we would look for the value of $p$ that minimizes $\mathbb{E}[L(\hat f_{p})]$.
Then, we would select $\hat f_{p}$ as our predictor.
However, in practice you do not have access to the expected loss, so you would need to estimate it using a test set, which is the topic of the next section.

# Part 2: Picking the Best Polynomial Predictor

In practice we only have access to a single dataset $\cal{D}$.
So we can only talk about the predictors $\hat f_p$ learned from this dataset.
As such, we will work with the expected loss $L(\hat f_p)$ of our predictor, instead of its expected value across all datasets $\mathbb{E}[L(\hat f_{D, p})]$.
<!-- We would like to plot an estimate of $\mathbb{E}[L(\hat f_{D, p})]$ so that we can select the polynomial degree $p$ that gives the minimum expected loss. -->
<!-- To do this we will need to split our dataset into a training and test set -->
To get a good estimate of $L(\hat f_p)$ we will need to split our dataset into a training and test set.
$$\cal{D}_\text{train} = ((\boldsymbol{x}_1, y_1), \ldots, (\boldsymbol{x}_{n-m}, y_{n-m})),$$
$$\cal{D}_\text{test} = ((\boldsymbol{x}_{n-m+1}, y_{n-m+1}), \ldots, (\boldsymbol{x}_n, y_n)).$$
Our learner will only use the training set to learn the predictor $\hat f_p$ that minimizes
$\hat L_\text{train}(f_p) = \frac{1}{n-m} \sum_{i=1}^{n-m} \ell(f_p(x_i), y_i)$ over all $f_p \in \cal{F}_p$.
<!-- Using the training set we can calculate  -->
<!-- $$\hat L_\text{train}(\hat f_p) = \frac{1}{n-m} \sum_{i=1}^{n-m} \ell(\hat f_p(x_i), y_i)$$ -->
<!-- which is an estimate of $\mathbb{E}[\hat L(\hat f_{D, p})]$. -->
Using the test set we can then calculate
$$\hat L_\text{test}(\hat f_p) = \frac{1}{m} \sum_{i=n-m+1}^{n} \ell(\hat f_p(x_i), y_i)$$
which is a good estimate of $L(\hat f_{D, p})$.

You need to implement `split_dataset` which splits the dataset `X`, `Y` into a training set `X_train`, `Y_train` and test set `X_test`, `Y_test`.
Instead of specifying the size of the test set $m$, it is more common to specify what percentage of the dataset should be used for the training set.
The argument `train_size` represents this percentage.
If the percentage of the training set causes the size of the training set to be a fraction, then the training set should be rounded down to the nearest integer.
For example, if the dataset has 10 samples and `train_size` is 0.75, then the training set should have 7 samples and the test set should have 3 samples.
If `train_size=0`, then `X_train`, `Y_train` should be empty numpy arrays, while if `train_size=1`, then `X_test`, `Y_test` should be empty numpy arrays.

### Question 2.1

Implement `split_dataset`.

_Points:_ 6

In [ ]:
def split_dataset(X, Y, train_size=0.5):
    """
    Splits the dataset into training and test sets.

    Parameters:
    X (numpy.ndarray): Feature matrix.
    Y (numpy.ndarray): Target vector.
    train_size (float): Proportion of the dataset to include in the training set.

    Returns:
    X_train (numpy.ndarray): Training feature matrix.
    Y_train (numpy.ndarray): Training target vector.
    X_test (numpy.ndarray): Test feature matrix.
    Y_test (numpy.ndarray): Test target vector.
    """

    ### YOUR CODE HERE ###

    X_train = X # this is a placeholder, replace it with the correct value
    Y_train = Y # this is a placeholder, replace it with the correct value

    X_test = X # this is a placeholder, replace it with the correct value
    Y_test = Y # this is a placeholder, replace it with the correct value

    ######################

    return X_train, Y_train, X_test, Y_test

In [ ]:
grader.check("q2_1")

First, lets assume we still have the ability to compute the expected loss $L(\hat f_p)$, and see how our estimates $\hat L_\text{train}(\hat f_p)$ and $\hat L_\text{test}(\hat f_p)$ compare.

The first slider for $n$ is the size of the dataset.
While the second slider "random seed" is the random seed used to generate the dataset of size $n$.
Changing the random seed should be thought of as getting an entirely new dataset of size $n$.
We have used `train_size = 0.5` for the split.

In the plot below you should see that for most datasets (i.e. random seed values) the test loss is a better estimate of the expected loss than the training loss.
You should also see that increasing $n$ causes both the training and test loss to become better estimates of the expected loss, as expected.
Since we only have access to one dataset we could not plot the expected values of the losses across datasets.
In particular, we could not plot $\mathbb{E}[L(\hat f_{D, p})]$ and $\mathbb{E}[\hat L(\hat f_{D, p})]$, but instead had to plot their estimates based on a single dataset split into a training and test set.
This is why for most datasets the behaviour of $\hat L_\text{test}(\hat f_p)$ resembles that of $\mathbb{E}[L(\hat f_{D, p})]$ from the previous plot.
Similarly $\hat L_\text{test}(\hat f_p)$ resembles the behaviour of $\mathbb{E}[\hat L(\hat f_{D, p})]$.
However, there are some datasets where the behaviour appears different, which is expected since we are only looking at a single dataset.

In [ ]:
# @title Plot

def update_plot_sample_loss_all(seed, n, show_plot):
    if show_plot:
        np.random.seed(seed)

        # Number of data points
        # n = 60

        # Generate X values uniformly between 0 and 4
        X_raw = np.random.uniform(0, 4, n)
        X = np.c_[np.ones(n), X_raw]

        # Generate Y values
        noise = np.random.normal(0, 1, n)
        Y = f_bayes(X_raw) + noise

        X_train, Y_train, X_test, Y_test = split_dataset(X, Y, train_size=0.5)

        # Calculate the loss for each polynomial degree using closed_form_learner
        L_f_bayes = estimated_loss(f_bayes, X_exp, Y_exp)
        L_f_star = []
        L_f_hat = []
        L_hat_f_hat = []
        L_hat_test_f_hat = []

        for degree in degrees:
            coeffs = np.polyfit(X_exp, Y_exp, degree)
            f_star = lambda x: np.polyval(coeffs, x)

            # Calculate the estimated loss
            L = estimated_loss(f_star, X_exp, Y_exp)
            L_f_star.append(L)

            # Transform the feature matrix to polynomial features
            X_train_poly = phi_p(X_train, degree)

            # Fit the model using closed form solution
            f_hat, w_hat = closed_form_learner(X_train_poly, Y_train)

            # Calculate the estimated loss on the training data
            L_hat = estimated_loss(f_hat, X_train_poly, Y_train)
            L_hat_f_hat.append(L_hat)

            X_poly = phi_p(np.c_[np.ones(X_exp.shape[0]), X_exp], degree)
            L = estimated_loss(f_hat, X_poly, Y_exp)
            L_f_hat.append(L)

            X_test_poly = phi_p(X_test, degree)
            L_hat_test = estimated_loss(f_hat, X_test_poly, Y_test)
            L_hat_test_f_hat.append(L_hat_test)

        # Plot the estimated loss as a function of polynomial degrees and the closed form learner loss
        plt.figure(figsize=(10, 6))
        plt.plot(degrees, L_f_star, color='green', marker='o', label=r'$L(f^*_p)$', linewidth=3, markersize=10)
        plt.plot(degrees, L_f_hat, color='orange', marker='x', linestyle='--', label=r'$L(\hat{f}_p)$', linewidth=3, markersize=10)
        plt.plot(degrees, L_hat_f_hat, color='purple', marker='x', linestyle='--', label=r'$\hat{L}_{train}(\hat{f}_p)$', linewidth=3, markersize=10)
        plt.plot(degrees, L_hat_test_f_hat, color = 'blue', marker='x', linestyle='--', label=r'$\hat{L}_{test}(\hat{f}_p)$', linewidth=3, markersize=10)
        plt.axhline(y=L_f_bayes, color='black', linestyle='-', label=r'$L(f_{Bayes})$', linewidth=3)
        plt.xlabel('Polynomial Degree p')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a slider for the variable see and a checkbox for showing/hiding the plot
seed_slider_sample_loss_all = widgets.IntSlider(value=1, min=1, max=10, step=1, description='random seed', style={'description_width': 'initial'})
n_slider_sample_loss_all = widgets.IntSlider(value=60, min=10, max=500, step=5, description='n')
show_plot_checkbox_sample_loss_all = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to update the plot based on the slider and checkbox values
interactive_plot_sample_loss_all = widgets.interactive_output(update_plot_sample_loss_all, {'n': n_slider_sample_loss_all, 'seed': seed_slider_sample_loss_all, 'show_plot': show_plot_checkbox_sample_loss_all})

# Display the slider, checkbox, and the plot
display(n_slider_sample_loss_all, seed_slider_sample_loss_all, show_plot_checkbox_sample_loss_all, interactive_plot_sample_loss_all)

Let us now simulate the process of selecting the best predictor.
In practice we only have access to $\hat L_{\text{train}}(\hat f_p)$ and $\hat L_{\text{test}}(\hat f_p)$ for a single dataset of size $n$.
In this case we set `n=100` and split the traning and test set evenly `train_size=0.5`.

First, for each value of $p \in \{1, \dots, 10\}$ we use `closed_form_learner` only on the training set with the function class $\cal{F}_p$ to get 10 different predictors $\hat f_1, \dots, \hat f_{10}$.
Now, we need to select the best predictor predictor from $\hat f_1, \dots, \hat f_{10}$.
To do this we plot $\hat L_{\text{train}}(\hat f_p)$ and $\hat L_{\text{test}}(\hat f_p)$.
You should see the training loss $\hat L_{\text{train}}(\hat f_p)$ decreases as $p$ increases, indicating that our `closed_form_learner` is better able to fit the training data as $p$ increases.
However, we want to choose the predictor that minimizes the expected loss, which we saw is better estimated by the test loss $\hat L_{\text{test}}(\hat f_p)$.
Thus, based on the plot below, we would select $\hat f_4$ as our predictor, since it has the lowest test loss.

In [ ]:
# @title Plot

# Initialize variables to store the predictors
predictor_degree_2 = None
predictor_degree_4 = None
predictor_degree_10 = None

def update_plot_sample_loss_fixed(show_plot):
    global predictor_degree_2, predictor_degree_4, predictor_degree_10

    if show_plot:
        seed = 2
        n = 100
        np.random.seed(seed)

        # Generate X values uniformly between 0 and 4
        X_raw = np.random.uniform(0, 4, n)
        X = np.c_[np.ones(n), X_raw]

        # Generate Y values
        noise = np.random.normal(0, 1, n)
        Y = f_bayes(X_raw) + noise

        X_train, Y_train, X_test, Y_test = split_dataset(X, Y, train_size=0.5)

        # Calculate the loss for each polynomial degree using closed_form_learner
        L_hat_f_hat = []
        L_hat_test_f_hat = []

        for degree in degrees:
            # Transform the feature matrix to polynomial features
            X_train_poly = phi_p(X_train, degree)

            # Fit the model using closed form solution
            f_hat, w_hat = closed_form_learner(X_train_poly, Y_train)

            # Save the predictors for degrees 2, 4, and 10
            if degree == 2:
                predictor_degree_2 = f_hat
            elif degree == 4:
                predictor_degree_4 = f_hat
            elif degree == 10:
                predictor_degree_10 = f_hat

            # Calculate the estimated loss on the training data
            L_hat = estimated_loss(f_hat, X_train_poly, Y_train)
            L_hat_f_hat.append(L_hat)

            X_test_poly = phi_p(X_test, degree)
            L_hat_test = estimated_loss(f_hat, X_test_poly, Y_test)
            L_hat_test_f_hat.append(L_hat_test)

        # Plot the estimated loss as a function of polynomial degrees and the closed form learner loss
        plt.figure(figsize=(10, 6))
        plt.plot(degrees, L_hat_f_hat, color='purple', marker='x', linestyle='--', label=r'$\hat{L}_{train}(\hat{f}_p)$', linewidth=3, markersize=10)
        plt.plot(degrees, L_hat_test_f_hat, color = 'blue', marker='x', linestyle='--', label=r'$\hat{L}_{test}(\hat{f}_p)$', linewidth=3, markersize=10)
        plt.xlabel('Polynomial Degree p')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a checkbox for showing/hiding the plot
show_plot_checkbox_sample_loss_fixed = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to update the plot based on the checkbox value
interactive_plot_sample_loss_fixed = widgets.interactive_output(update_plot_sample_loss_fixed, {'show_plot': show_plot_checkbox_sample_loss_fixed})

# Display the checkbox and the plot
display(show_plot_checkbox_sample_loss_fixed, interactive_plot_sample_loss_fixed)


We can visualize the predictor $\hat f_4$ by plotting it on the same plot as $f_\text{Bayes}$.
We can also compare it to predictors that have higher test loss, such as $\hat f_2$ and $\hat f_{10}$.

You should see that $\hat f_4$ is a better approximation of $f_\text{Bayes}$ than $\hat f_2$ and $\hat f_{10}$.

In [ ]:
# @title Plot

# Define the function to update the plot based on the selected degree, n, and random seed
def update_plot_best_f(show_plot):
    if show_plot:
        deg_2_values = predictor_degree_2(phi_p(np.c_[np.ones(x.shape[0]), x], 2))
        deg_4_values = predictor_degree_4(phi_p(np.c_[np.ones(x.shape[0]), x], 4))
        deg_10_values = predictor_degree_10(phi_p(np.c_[np.ones(x.shape[0]), x], 10))

        # Plot the generated data points and the polynomial fit
        plt.figure(figsize=(10, 6))
        plt.plot(x, f_bayes(x), color='black', label=r'$f_{Bayes}$', linewidth=3)
        plt.plot(x, deg_2_values, color='orange', linestyle='--', label=fr'$\hat f_2$', linewidth=3)
        plt.plot(x, deg_4_values, color='green', linestyle='-', label=fr'$\hat f_4$', linewidth=3)
        plt.plot(x, deg_10_values, color='blue', linestyle=':', label=r'$\hat{f}_{10}$', linewidth=3)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.ylim(0.2, 7.2)
        plt.xlim(-0.2, 4.2)
        plt.legend(loc='upper left')
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a checkbox for showing/hiding the plot
show_plot_checkbox_best_f = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to update the plot based on the checkbox value
interactive_plot_best_f = widgets.interactive_output(update_plot_best_f, {'show_plot': show_plot_checkbox_best_f})

# Display the checkbox and the plot
display(show_plot_checkbox_best_f, interactive_plot_best_f)

# Part 3: Regularization

In this section we will study the effect of regularization on our predictors.
To do this you will need to implement a regularized version of the batch gradient descent learner `regularized_bgd_learner`.
This learner should take its gradient steps with respect to the gradient of the regularized estimated loss
$$\hat L_\lambda(\boldsymbol{w}) = \frac{1}{n} \sum_{i=1}^n \ell(\boldsymbol{x_i}^\top \boldsymbol{w}, y_i) + \frac{\lambda}{n} \sum_{j=1}^{d} w_j^2.$$
For your implementation assume that the degree of the polynomial $p$ is 1.
This means you do not need to apply `phi_p` to the features `X` in `regularized_bgd_learner`.
The reason for this is that it makes the implementation simpler and it is without loss of generality since when we use your implementation to plot things we can simply apply `phi_p` first, before passing the features to `regularized_bgd_learner`.

### Question 3.1

Implement `regularized_bgd_learner`.

_Points:_ 12

In [ ]:
def regularized_bgd_learner(X, Y, step_size=0.01, lambda_=0.1, epochs=1000, random_seed=42):
    """
    Performs batch gradient descent with L2 regularization to learn the weights.

    Parameters:
    X (numpy.ndarray): Feature matrix of size (n, d+1), where n is the number of samples
                       and d is the number of features. The first column should be all 1s.
    Y (numpy.ndarray): Target vector of size (n, 1).
    alpha (float): Learning rate.
    lambda_ (float): Regularization parameter.
    epochs (int): Number of iterations for gradient descent.

    Returns:
    predictor (function): A function that takes a feature vector and returns a predicted value.
    w (numpy.ndarray): The learned weights.
    """
    n, d = X[:, 1:].shape
    np.random.seed(random_seed)
    w = np.random.randn(d+1) # initialize the weights randomly

    ### YOUR CODE HERE ###


    ######################

    def predictor(x):
        return x @ w

    return predictor, w

In [ ]:
grader.check("q3_1")

We plot the effect of regularization on the predictor `regularized_bgd_learner` outputs, for various values of $\lambda$.
We will use the function class $\cal{F}_9$, which contains polynomials of degree 9 or less.
When $\lambda = 0$ we should get almost the same predictor as the `closed_form_learner` (the only error is due to gradient descent potentially not being run for enough epochs).
Since the function class $\cal{F}_9$ is very large we should expect the predictor to overfit the data when $\lambda = 0$.
As $\lambda$ increases you should see that the predictor becomes less complicated and thus less likely to overfit the data.
When $\lambda$ gets too large you should see that the predictor becomes too simple and thus underfits the data.

We also plot the expected predictor $\bar f_9$ based on 10 datasets.
Note that we only used 10 datasets due to computational reasons.
Ideally we would use more datasets to get a better estimate of $\bar f_9$.
What you should see in the plot is that when $\lambda = 0$ the expected predictor $\bar f_9$ is approximately equal to $f_\text{Bayes}$ (it would be exactly equal if we used more datasets), which means the Bias is low.
As $\lambda$ increases $\bar f_9$ becomes less complicated and is no longer similar to $f_\text{Bayes}$, which means the Bias increases.
When $\lambda$ gets too large $\bar f_9$ becomes too simple and has high Bias.

You should also notice that for small $\lambda$ the predictor $\hat f_9$, varies a lot across datasets (random seed values) relative to $\bar f_9$, which means the Variance is high.
While for large $\lambda$ the predictor $\hat f_9$ varies less across datasets relative to $\bar f_9$, which means the Variance is low.

In [ ]:
# @title Plot

# Define the function to update the plot based on the selected random seed, lambda, and show_plot
def update_plot_f_bar_reg(random_seed, lambda_, show_plot):
    degree = 9
    n = 40
    if show_plot:
        # Fit the polynomial to the data
        w_star = np.polyfit(X_exp, Y_exp, degree)

        # Generate y values for the fitted polynomial
        f_star = np.polyval(w_star, x)

        # Generate new train dataset
        np.random.seed(random_seed)
        X_train_raw = np.random.uniform(0, 4, n)
        X_train = np.c_[np.ones(n), X_train_raw]
        noise_train = np.random.normal(0, 1, n)
        Y_train = f_bayes(X_train_raw) + noise_train

        # Transform the feature matrix to polynomial features
        X_train_poly = phi_p(X_train, degree)

        # Fit the model using regularized batch gradient descent
        f_hat, w_hat = regularized_bgd_learner(X_train_poly, Y_train, step_size=0.075, lambda_=lambda_, epochs=10000)

        # Generate y values for the fitted polynomial from regularized_bgd_learner
        f_hat_values = f_hat(phi_p(np.c_[np.ones(x.shape[0]), x], degree))

        # Calculate the average predictor
        num_datasets = 10
        f_bar_values = np.zeros_like(x)

        for _ in range(num_datasets):
            X_train_raw = np.random.uniform(0, 4, n)
            X_train = np.c_[np.ones(n), X_train_raw]
            noise_train = np.random.normal(0, 1, n)
            Y_train = f_bayes(X_train_raw) + noise_train

            X_train_poly = phi_p(X_train, degree)
            f_bar, w_bar = regularized_bgd_learner(X_train_poly, Y_train, step_size=0.075, lambda_=lambda_, epochs=10000)
            f_bar_values += f_bar(phi_p(np.c_[np.ones(x.shape[0]), x], degree))

        f_bar_values /= num_datasets

        # Plot the generated data points and the polynomial fit
        plt.figure(figsize=(10, 6))
        plt.plot(x, f_bayes(x), color='black', label=r'$f_{Bayes}$', linewidth=3)
        plt.plot(x, f_star, color='green', linestyle='-', label=fr'$f^*_{degree}$', linewidth=3)
        plt.plot(x, f_hat_values, color='orange', linestyle='--', label=fr'$\hat f_{degree}$ with n={n}', linewidth=3)
        plt.plot(x, f_bar_values, color='blue', linestyle='-', label=fr'$\bar f_{degree}$ over {num_datasets} datasets', linewidth=3)
        plt.scatter(X_train_raw, Y_train, color='orange', alpha=0.5, label='Datapoints')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.ylim(0.2, 7.2)
        plt.xlim(-0.2, 4.2)
        plt.legend(loc='upper left')
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create sliders for the random seed and lambda
random_seed_slider_f_bar_reg = widgets.IntSlider(value=1, min=1, max=10, step=1, description='Random Seed', style={'description_width': 'initial'})
lambda_slider_f_bar_reg = widgets.FloatLogSlider(
    value=0.01,
    base=10,
    min=-4,  # 10^-4 = 0.0001
    max=2,   # 10^1 = 10.0
    step=0.1,
    description='Lambda',
)
show_plot_checkbox_f_bar_reg = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to update the plot based on the slider values
interactive_plot_f_bar_reg = widgets.interactive_output(update_plot_f_bar_reg,
                                                    {'random_seed': random_seed_slider_f_bar_reg,
                                                     'lambda_': lambda_slider_f_bar_reg,
                                                     'show_plot': show_plot_checkbox_f_bar_reg})

# Display the sliders and the plot
display(random_seed_slider_f_bar_reg, lambda_slider_f_bar_reg, show_plot_checkbox_f_bar_reg, interactive_plot_f_bar_reg)


To conclude, we can plot training and test loss for various values of $\lambda$.
You should see that the training loss decreases as $\lambda$ increases, since it allows the predictor to become more complicated.
However, the test loss should at first decrease as $\lambda$ increases (bias is reducing a lot), but around $\lambda = 1$ it should start to increase (variance is increasing a lot).

Similar to [Part 2](#part-2-picking-the-best-polynomial-predictor), we would use the test loss to select the best predictor, which in this case would be the predictor output by `regularized_bgd_learner` with $\lambda$ slightly less than 1.

In [ ]:
# @title Plot

# Function to update the plot based on the checkbox value
def update_plot_reg(show_plot):
    if show_plot:
        # Number of data points
        n = 18

        # Set the random seed for reproducibility
        random_seed = 7
        np.random.seed(random_seed)

        X_raw = np.random.uniform(0, 4, n)
        X = np.c_[np.ones(n), X_raw]

        # Generate noise from a Gaussian distribution with mean 0 and variance 1
        noise_train = np.random.normal(0, 1, n)

        # Calculate Y values using f_bayes
        Y = f_bayes(X_raw) + noise_train

        X_train, Y_train, X_test, Y_test = split_dataset(X, Y, train_size=0.5)

        # Define the range of lambda values to test
        lambda_values = np.logspace(2, -2, 10)

        # Initialize lists to store the losses
        L_hat_f_hat_lambda = []
        L_hat_test_f_hat_lambda = []

        # Degree of polynomial features
        degree = 9

        # Train and evaluate the model for each lambda value
        for lambda_ in lambda_values:
            # Transform the feature matrix to polynomial features
            X_train_poly = phi_p(X_train, degree)
            X_test_poly = phi_p(X_test, degree)

            # Fit the model using regularized batch gradient descent
            f_hat_lambda, w_hat_lambda = regularized_bgd_learner(X_train_poly, Y_train, step_size=0.01, lambda_=lambda_, epochs=10000)

            # Calculate the estimated loss on the training data
            L_hat = estimated_loss(f_hat_lambda, X_train_poly, Y_train)
            L_hat_f_hat_lambda.append(L_hat)

            # Calculate the estimated loss on the test data
            L_hat_test = estimated_loss(f_hat_lambda, X_test_poly, Y_test)
            L_hat_test_f_hat_lambda.append(L_hat_test)

        # Plot the estimated loss as a function of lambda values
        plt.figure(figsize=(10, 6))
        plt.plot(lambda_values, L_hat_f_hat_lambda, color='purple', marker='x', linestyle='--', label=r'$\hat{L}_{train, \lambda}(\hat{f})$', linewidth=3, markersize=10)
        plt.plot(lambda_values, L_hat_test_f_hat_lambda, color = 'blue', marker='x', linestyle='--', label=r'$\hat{L}_{test, \lambda}(\hat{f})$', linewidth=3, markersize=10)
        plt.xscale('log')
        plt.gca().invert_xaxis()  # Invert the x-axis
        plt.xlabel(r'$\lambda$')
        plt.ylim(0.2, 2.7)
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a checkbox for showing/hiding the plot
show_plot_checkbox_reg = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to update the plot based on the checkbox value
interactive_plot_reg = widgets.interactive_output(update_plot_reg, {'show_plot': show_plot_checkbox_reg})

# Display the checkbox and the plot
display(show_plot_checkbox_reg, interactive_plot_reg)


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()